In [1]:
from pymongo import MongoClient
import json
import numpy as np
import pandas as pd
import pymongo
import time
import math
from pymongo import ReadPreference
 
### @ IP 
# 34.196.21.20
# port:27017

#34.197.19.183:27017

In [35]:
#config = {'_id': 'rs0', 'members': [
#        {'_id': 0, 'host': '34.196.21.20:27017'},
#        {'_id': 1, 'host': '34.197.19.183:27018'},
#        {'_id': 2, 'host': '34.193.159.184:27019'}]}
#client4.admin.command("replSetInitiate", config)


In [2]:
client4 = MongoClient('34.196.21.20',27017, replicaset='rs0')

In [3]:
print(client4.nodes)

frozenset()


In [4]:
print(client4.read_preference)

Primary()


In [5]:
client4.database_names() 

['admin', 'election', 'local']

In [6]:
db = client4['election']   
db.collection_names()

['grdelec', 'us']

In [6]:
# nom des colonnes dans la collection us: vote_id, vote_timestamp, state, vote_result

In [7]:
db.us.find_one()

{'_id': ObjectId('587d1b4e2690f54cd18630ff'),
 'state': 'Minnesota',
 'vote_id': 24,
 'vote_result': 'Clinton',
 'vote_timestamp': '2016-11-08T20:00'}

In [8]:
# db.us.find({"vote_result": "Clinton"}).count() # ca marche mais c'est long

In [9]:
# listes 
liste_state = ['Minnesota', 'Alabama', 'Montana', 'Utah', 'Caroline_du_Sud', 'New_York', 'Nouveau_Mexique', 'Californie', 'Colorado', 'Connecticut', 'District_de_Columbia', 'Kentucky', 'Caroline_du_Nord', 'Vermont', 'Dakota_du_Sud', 'Alaska', 'Arkansas', 'Washington', 'Hawai', 'Wisconsin', 'Georgie', 'Ohio', 'Missouri', 'New_Hampshire', 'Virginie', 'Iowa', 'Delaware', 'Arizona', 'Louisiane', 'Massachusetts', 'Texas', 'Dakota_du_Nord', 'Tennessee', 'Wyoming', 'Floride', 'Illinois', 'Nevada', 'Idaho', 'Pennsylvanie', 'Virginie_Occidentale', 'Maryland', 'Mississippi', 'Rhode_Island', 'Michigan', 'Oklahoma', 'Nebraska', 'New_Jersey', 'Indiana', 'Kansas', 'Maine', 'Oregon']
liste_candidate = ['Autre', 'Blanc', 'Castle', 'Clinton', 'Johnson', 'McMullin', 'Stein', 'Trump']
liste_time = ['2016-11-08T20:00', '2016-11-08T20:01', '2016-11-08T20:02', '2016-11-08T20:03', '2016-11-08T20:04', '2016-11-08T20:05', '2016-11-08T20:08', '2016-11-08T20:09', '2016-11-08T20:10', '2016-11-08T20:11', '2016-11-08T20:13', '2016-11-08T20:15', '2016-11-08T20:17', '2016-11-08T20:19', '2016-11-08T20:22', '2016-11-08T20:24', '2016-11-08T20:27', '2016-11-08T20:28', '2016-11-08T20:29', '2016-11-08T20:31', '2016-11-08T20:34', '2016-11-08T20:35', '2016-11-08T20:36', '2016-11-08T20:37', '2016-11-08T20:40', '2016-11-08T20:41', '2016-11-08T20:43', '2016-11-08T20:48', '2016-11-08T20:50', '2016-11-08T20:51', '2016-11-08T20:52', '2016-11-08T20:56', '2016-11-08T20:57', '2016-11-08T20:58', '2016-11-08T20:59']

## Functions

In [ ]:
# new fct Transformation du resultat d'une requete en Dataframe
def key_time_dataframe(key_time):
    cursor=db.us.aggregate([
                 { "$match": { "vote_timestamp":key_time } },
                 { "$group": { "_id": { "state": "$state", "vote_result":"$vote_result"}, "total":  {"$sum":1 }}  }, 
               ])
    info=(list(cursor)) # type dico
    ligne=[]
    df = pd.DataFrame() 
    for i in range(len(info)):
        ligne=[info[i]['total'] ,info[i]['_id']['state'],info[i]['_id']['vote_result']]
        df = pd.concat([df,pd.DataFrame(ligne).T]) 
    df.columns=['total','state','vote_result']
    return df

In [10]:
# Transformation du resultat d'une requete en Dataframe

def query_to_dataframe(cursor):
    df = pd.DataFrame()
    if 'result' in cursor.keys():
        for i, el in enumerate(cursor['result']):
            df = df.append(pd.DataFrame(el, index=[i]), ignore_index=True)
    return df

In [11]:
# Requete Laura

def query_v2(key_time):
    cursor=db.us.aggregate([
                { "$match": { "vote_timestamp":key_time } },
                { "$group": { "_id": { "state": "$state", "vote_result":"$vote_result"}, "total":  {"$sum":1 }}  },
              ])
    info=(list(cursor))
    ligne=[]
    df=pd.DataFrame()
    for i in range(len(info)):
        ligne=[info[i]['total'] ,info[i]['_id']['state'],info[i]['_id']['vote_result']]
        df=pd.concat([df,pd.DataFrame(ligne).T])
    df.columns=['nb_vote','state','result']

    return df

In [33]:
# Requete nombre de votes par Candidat par Etat 

def query_by_time(key_time):  
    cursor = db.us.aggregate([
                 { "$match": { "vote_timestamp":key_time } },
                 { "$group": { "_id": { "state": "$state", "vote_result": "$vote_result" } 
                              , "total":  {"$sum":1 }}  }, 
               ]) 
    return test

In [49]:
cursor =db.us.aggregate([
         { "$match": { "vote_timestamp":key_time } },
         { "$group": { "_id": { "state": "$state", "vote_result":"$vote_result"}, "total":  {"$sum":1 }}  }, 
       ])
print(list(cursor))

[{'total': 53076, '_id': {'vote_result': 'McMullin', 'state': 'Minnesota'}}, {'total': 36985, '_id': {'vote_result': 'Stein', 'state': 'Minnesota'}}, {'total': 41657, '_id': {'vote_result': 'Autre', 'state': 'Minnesota'}}, {'total': 1322951, '_id': {'vote_result': 'Trump', 'state': 'Minnesota'}}, {'total': 112972, '_id': {'vote_result': 'Johnson', 'state': 'Minnesota'}}, {'total': 9456, '_id': {'vote_result': 'Castle', 'state': 'Minnesota'}}, {'total': 1367716, '_id': {'vote_result': 'Clinton', 'state': 'Minnesota'}}]


In [13]:
# Requete du nombre de Grands Electeurs par State

def query_grands_elec():
    df_grd_elec = pd.DataFrame()
    cur = db.grdelec.find()
    xcur = list(cur)
    if len(xcur):
        for j, elt in enumerate(xcur):
            df_grd_elec = df_grd_elec.append(pd.DataFrame(elt, index=[j]), ignore_index=True)
    return df_grd_elec

In [46]:
cursor = query_by_time(key_time) 
type(cursor)

pymongo.command_cursor.CommandCursor

In [72]:
# Resultat votes par State

def states_result(key_time):
    
#     cursor = query_by_time(key_time)
#     df_state = query_to_dataframe(cursor)
    df_state = key_time_dataframe(key_time)
    df_resultat = pd.DataFrame()
    df_prop_state = pd.DataFrame()
    
    if not df_state.empty:
        df_state_group = df_state.groupby('state')
        for key, elmt in df_state_group:
            elmt['pct'] = elmt['total'].apply(lambda x: x/elmt.total.sum())
            df_resultat = df_resultat.append(elmt[elmt.pct == elmt.pct.max()][['state', 'vote_result', 'total']], ignore_index=True)

            df_prop_state = df_prop_state.append(elmt[elmt.state == 'Maine'], ignore_index=True)
            df_prop_state = df_prop_state.append(elmt[elmt.state == 'Nebraska'], ignore_index=True)
    
    return df_resultat, df_prop_state

In [69]:
df_state = key_time_dataframe(key_time)
df_resultat = pd.DataFrame()
df_prop_state = pd.DataFrame()

if not df_state.empty:
    df_state_group = df_state.groupby('state')
    for key, elmt in df_state_group:
        elmt['pct'] = elmt['total'].apply(lambda x: x/elmt.total.sum())
        df_resultat = df_resultat.append(elmt[elmt.pct == elmt.pct.max()][['state', 'vote_result', 'total']], ignore_index=True)

        df_prop_state = df_prop_state.append(elmt[elmt.state == 'Maine'], ignore_index=True)
        df_prop_state = df_prop_state.append(elmt[elmt.state == 'Nebraska'], ignore_index=True)
    

C:\Users\laura\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
states_result(key_time)[0]

C:\Users\laura\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,state,vote_result,total
0,Minnesota,Clinton,1367716


In [61]:
# Resultat de l'élection

def score_election(df_r, df_grd_elec):
    result = pd.merge(df_r, df_grd_elec, on='state')
    return result[['vote_result', 'grands_electeurs']].groupby('vote_result').sum()

In [62]:
# Resultat de l'élection par State

def score_state(df_r, df_grd_elec):
    return pd.merge(df_r, df_grd_elec, on='state')

In [63]:
# Maine et le Nebraska

def score_election_prop(df_r, df_prop_state, df_grd_elec):
    result = pd.merge(df_r, df_grd_elec, on='state')
    result[['vote_result', 'grands_electeurs']].groupby('vote_result').sum()

In [64]:
test = query_by_time(key_time)
print(len(list(test)))

7


In [65]:
test = query_by_time(key_time)
print(list(test)[0])

{'total': 53076, '_id': {'vote_result': 'McMullin', 'state': 'Minnesota'}}


## Fonction de Test

In [82]:
start_time = time.time()
key_time = '2016-11-08T20:00'
df_r = pd.DataFrame()
df_prop_state = pd.DataFrame()

# Boucle de simulation de l'incrémentation du temps
for el in range(61):
    try:
        # CRéation du timestamp format string mongodb
        key_time = '2016-11-08T20:' + ('00' + str(el))[-2:]
        print(key_time)
        
        # Recupération des votes + stockage dans un Dataframe (df_r)
        df_r = df_r.append(states_result(key_time)[0], ignore_index=True)
        
        # Recupération des votes pour le calcul des états au scrutin proportionnel + stockage dans un Dataframe
        df_prop_state = df_prop_state.append(states_result(key_time)[1], ignore_index=True)
        print(df_r)
        
    except ValueError:# pour les minutes ou il n'y a pas de candidats sinon bouclé sur la liste de time_stamp
        print(df_r)

print(df_prop_state)
print("--- %s seconds ---" % (time.time() - start_time))  

2016-11-08T20:00


C:\Users\laura\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


       state vote_result    total
0  Minnesota     Clinton  1367716
2016-11-08T20:01
       state vote_result    total
0  Minnesota     Clinton  1367716
1    Alabama       Trump  1318255
2    Montana       Trump   279240
3       Utah       Trump   515231
2016-11-08T20:02
             state vote_result    total
0        Minnesota     Clinton  1367716
1          Alabama       Trump  1318255
2          Montana       Trump   279240
3             Utah       Trump   515231
4  Caroline_du_Sud       Trump  1155389
5         New_York     Clinton  4547418
2016-11-08T20:03
             state vote_result    total
0        Minnesota     Clinton  1367716
1          Alabama       Trump  1318255
2          Montana       Trump   279240
3             Utah       Trump   515231
4  Caroline_du_Sud       Trump  1155389
5         New_York     Clinton  4547418
6  Nouveau_Mexique     Clinton   385234
2016-11-08T20:04
             state vote_result    total
0        Minnesota     Clinton  1367716
1          Ala

In [83]:
df_grd_elec = query_grands_elec()

print(df_grd_elec.head())
print(df_grd_elec['grands_electeurs'].sum())

                        _id code  grands_electeurs            state
0  58823c757c12b81956bc3d8f   AZ                11          Arizona
1  58823c757c12b81956bc3d93   SC                 9  Caroline_du_Sud
2  58823c757c12b81956bc3da1   KS                 6           Kansas
3  58823c757c12b81956bc3da2   KY                 8         Kentucky
4  58823c757c12b81956bc3db0   NY                29         New_York
538


In [84]:
print(score_election(df_r, df_grd_elec))

             grands_electeurs
vote_result                  
Clinton                   233
Trump                     305


En cours....

In [135]:
# Calcul grands electeurs %proportionnel Maine et le Nebraska

result_prop = pd.merge(df_prop_state, df_grd_elec, on='state')
df_inter = result_prop.groupby('state')
df_grd_elec_fin = pd.DataFrame()

for key, elmt in df_inter:
    df_agg = elmt.sort_values('pct', ascending=False)[:2]
    df_agg['grands_electeurs'] = (df_agg['pct'] * df_agg['grands_electeurs']).apply(lambda x: math.ceil(x))
    df_grd_elec_fin = df_grd_elec_fin.append(df_agg, ignore_index=True)


print(df_grd_elec_fin[['state', 'vote_result', 'code', 'grands_electeurs']])

test = df_grd_elec_fin

#result[(result.state == 'Nebraska') | (result.state == 'Maine')]

test = test.drop(test[(test.state == 'Nebraska') | (test.state == 'Maine')].index.tolist())
print(test[(test.state == 'Nebraska') | (test.state == 'Maine')])
fin = test.append(df_grd_elec_fin[['state', 'vote_result', 'code', 'grands_electeurs']], ignore_index=True)
print()
print(fin[(fin.state == 'Nebraska') | (fin.state == 'Maine')])
print()
print(fin[['vote_result', 'grands_electeurs']].groupby('vote_result').sum())

      state vote_result code  grands_electeurs
0     Maine     Clinton   ME                 2
1     Maine       Trump   ME                 2
2  Nebraska       Trump   NE                 3
3  Nebraska     Clinton   NE                 2
Empty DataFrame
Columns: [_id_x, state, total, vote_result, pct, _id_y, code, grands_electeurs]
Index: []

   _id_x _id_y code  grands_electeurs  pct     state  total vote_result
0    NaN   NaN   ME                 2  NaN     Maine    NaN     Clinton
1    NaN   NaN   ME                 2  NaN     Maine    NaN       Trump
2    NaN   NaN   NE                 3  NaN  Nebraska    NaN       Trump
3    NaN   NaN   NE                 2  NaN  Nebraska    NaN     Clinton

             grands_electeurs
vote_result                  
Clinton                     4
Trump                       5
